In [1]:
!pip install ipython-sql
%load_ext sql
import os
USER = os.environ['POSTGRESQL_USER']
PASSWORD = os.environ['POSTGRESQL_PASSWORD']
POSTGRESQL_HOST = '10.129.0.25'
DBASE_NAME = 'demo'
CONNECT_DATA = 'postgresql://{}:{}@{}/{}'.format(
    USER,
    PASSWORD,
    POSTGRESQL_HOST,
    DBASE_NAME
)

In [2]:
%%sql $CONNECT_DATA
SELECT distinct arrival_airport, aircraft_code FROM flights f
WHERE f.arrival_airport='AER'


7 rows affected.


arrival_airport,aircraft_code
AER,773
AER,733
AER,763
AER,CN1
AER,SU9
AER,CR2
AER,319


**№1 Which fare conditions are available for flights from AER to SVO (i.e. from Adler/Sochi to Sheremetevo/Moscow)**

In [ ]:
#Solution without JOINS

In [3]:
%%sql $CONNECT_DATA
    SELECT distinct fare_conditions as Available_fare_conditions_from_AER_to_SVO 
    FROM Ticket_flights 
    WHERE flight_id in (SELECT flight_id FROM Flights
    WHERE departure_airport = 'AER' and arrival_airport = 'SVO')


3 rows affected.


available_fare_conditions_from_aer_to_svo
Business
Comfort
Economy


In [ ]:
#Solution with JOINS

In [4]:
%%sql $CONNECT_DATA
    SELECT distinct departure_airport, arrival_airport, fare_conditions as Available_fare_conditions_from_AER_to_SVO 
    FROM Ticket_flights JOIN
    (SELECT departure_airport, arrival_airport, flight_id FROM Flights
    WHERE departure_airport = 'AER' and arrival_airport = 'SVO') as n
    ON Ticket_flights.flight_id = n.flight_id


3 rows affected.


departure_airport,arrival_airport,available_fare_conditions_from_aer_to_svo
AER,SVO,Business
AER,SVO,Comfort
AER,SVO,Economy


**№2 A list of aircraft models which fly to the Yakutsk AND Abakan cities**

In [ ]:
#Solution without JOINS

In [40]:
%%sql $CONNECT_DATA
SELECT model as aircraft_models_which_fly_to_the_Yakutsk_AND_Abakan FROM
       Aircrafts
    WHERE aircraft_code in (SELECT distinct aircraft_code 
                            FROM Flights
                            WHERE arrival_airport = (SELECT distinct airport_code 
                                                      FROM Airports
                                                      WHERE city = 'Yakutsk'))
           AND
               aircraft_code in (SELECT distinct aircraft_code
                            FROM Flights
                            WHERE arrival_airport = (SELECT distinct airport_code 
                                                      FROM Airports
                                                      WHERE city = 'Abakan'))
                

1 rows affected.


aircraft_models_which_fly_to_the_yakutsk_and_abakan
Airbus A319-100


In [ ]:
#Solution with JOINS

In [36]:
%%sql $CONNECT_DATA
    SELECT arrival_airport_Yakutsk, arrival_airport_Abakan, n.aircraft_code, model as aircraft_models_which_fly_to_the_Yakutsk_AND_Abakan
    FROM Aircrafts
    JOIN
     (SELECT arrival_airport_Yakutsk,arrival_airport_Abakan, a.aircraft_code FROM
     (SELECT distinct aircraft_code, arrival_airport as arrival_airport_Yakutsk 
                            FROM Flights
                            WHERE arrival_airport = (SELECT distinct airport_code 
                                                      FROM Airports
                                                      WHERE city = 'Yakutsk')) as a
           JOIN
                (SELECT distinct aircraft_code, arrival_airport as arrival_airport_Abakan 
                            FROM Flights
                            WHERE arrival_airport = (SELECT distinct airport_code 
                                                      FROM Airports
                                                      WHERE city = 'Abakan')) as b
            ON a.aircraft_code = b.aircraft_code) as n
        ON Aircrafts.aircraft_code = n.aircraft_code

1 rows affected.


arrival_airport_yakutsk,arrival_airport_abakan,aircraft_code,aircraft_models_which_fly_to_the_yakutsk_and_abakan
YKS,ABA,319,Airbus A319-100


**№ 3 A list of aircraft models which fly to the Yakutsk OR Abakan cities**

In [ ]:
#Solution without JOINS

In [79]:
%%sql $CONNECT_DATA
    SELECT model as aircraft_models_which_fly_to_the_Yakutsk_OR_Abakan
    FROM Aircrafts
    WHERE aircraft_code in (SELECT distinct aircraft_code 
                            FROM Flights
                            WHERE arrival_airport = (SELECT distinct airport_code 
                                                      FROM Airports
                                                      WHERE city = 'Yakutsk'))
           OR
               aircraft_code in (SELECT distinct aircraft_code 
                            FROM Flights
                            WHERE arrival_airport = (SELECT distinct airport_code 
                                                      FROM Airports
                                                      WHERE city = 'Abakan'))

5 rows affected.


aircraft_models_which_fly_to_the_yakutsk_or_abakan
Sukhoi Superjet-100
Airbus A319-100
Boeing 737-300
Cessna 208 Caravan
Bombardier CRJ-200


In [ ]:
#Solution with JOINS - NOTE: We cannot extarct the duplicate aircraft_code = 319 there because we have unique (distinct) 
#concationation of arrival_airport + aircraft code. We should either wipe out arrival airport from the output or accept a duplicate.

In [77]:
%%sql $CONNECT_DATA
        SELECT arrival_airport, n.aircraft_code, model as aircraft_models_which_fly_to_the_Yakutsk_OR_Abakan
    FROM Aircrafts
    JOIN
     (SELECT distinct aircraft_code, arrival_airport 
                            FROM Flights
                            WHERE arrival_airport = (SELECT distinct airport_code 
                                                      FROM Airports
                                                      WHERE city = 'Yakutsk') 
           UNION
                (SELECT distinct aircraft_code, arrival_airport
                            FROM Flights
                            WHERE arrival_airport = (SELECT distinct airport_code 
                                                      FROM Airports
                                                      WHERE city = 'Abakan'))) as n
        ON Aircrafts.aircraft_code = n.aircraft_code


6 rows affected.


arrival_airport,aircraft_code,aircraft_models_which_fly_to_the_yakutsk_or_abakan
ABA,733,Boeing 737-300
ABA,319,Airbus A319-100
ABA,CN1,Cessna 208 Caravan
YKS,SU9,Sukhoi Superjet-100
YKS,319,Airbus A319-100
YKS,CR2,Bombardier CRJ-200


**№ 4 A list of aircrafts which DON’T fly to Yakutsk city**

In [ ]:
#Solution without JOINS

In [78]:
%%sql $CONNECT_DATA
    SELECT model as aircraft_models_which_do_not_fly_to_Yakutsk
    FROM Aircrafts
    WHERE aircraft_code not in (SELECT distinct aircraft_code 
                            FROM Flights
                            WHERE arrival_airport = (SELECT distinct airport_code 
                                                      FROM Airports
                                                      WHERE city = 'Yakutsk'))

6 rows affected.


aircraft_models_which_do_not_fly_to_yakutsk
Boeing 777-300
Boeing 767-300
Airbus A320-200
Airbus A321-200
Boeing 737-300
Cessna 208 Caravan


In [ ]:
#additional check - A list of aircrafts which DO fly to Yakutsk city

In [84]:
%%sql $CONNECT_DATA
    SELECT model as aircraft_models_which_do_fly_to_Yakutsk
    FROM Aircrafts
    WHERE aircraft_code in (SELECT distinct aircraft_code 
                            FROM Flights
                            WHERE arrival_airport = (SELECT distinct airport_code 
                                                      FROM Airports
                                                      WHERE city = 'Yakutsk'))

3 rows affected.


aircraft_models_which_do_fly_to_yakutsk
Sukhoi Superjet-100
Airbus A319-100
Bombardier CRJ-200


In [ ]:
#Solution with JOIN

In [8]:
%%sql $CONNECT_DATA
    SELECT f.aircraft_code, model as aircraft_models_which_do_not_fly_to_Yakutsk
    FROM Aircrafts
    FULL OUTER JOIN (SELECT distinct aircraft_code 
                            FROM Flights 
                                JOIN (SELECT distinct airport_code 
                                                      FROM Airports
                                                      WHERE city = 'Yakutsk') as g
                               ON Flights.arrival_airport = g.airport_code) as f
    ON Aircrafts.aircraft_code = f.aircraft_code

9 rows affected.


aircraft_code,aircraft_models_which_do_not_fly_to_yakutsk
None,Boeing 777-300
None,Boeing 767-300
SU9,Sukhoi Superjet-100
None,Airbus A320-200
None,Airbus A321-200
319,Airbus A319-100
None,Boeing 737-300
None,Cessna 208 Caravan
CR2,Bombardier CRJ-200


**№ 5 Find out airport codes related to Mrs. 'ELLA DMITRIEVA'  - from where and to where she flew.**

In [ ]:
#Sloution without JOINS

In [77]:
%%sql $CONNECT_DATA
SELECT departure_airport as departure_airport_of_Ms_Ella_Dmitirieva, 
arrival_airport as arrival_airport_of_Ms_Ella_Dmitirieva
FROM Flights
WHERE flight_id in (SELECT flight_id 
                    FROM Ticket_flights
                    WHERE ticket_no in (SELECT ticket_no 
                                        FROM Tickets
                                        WHERE passenger_name = 'ELLA DMITRIEVA'))

7 rows affected.


departure_airport_of_ms_ella_dmitirieva,arrival_airport_of_ms_ella_dmitirieva
SVO,SVX
MQF,SVX
SVX,MQF
SVX,SVO
DME,HMA
DME,UUS
HMA,DME


In [4]:
%%sql $CONNECT_DATA
SELECT departure_airport as departure_airport_of_Ms_Ella_Dmitirieva, 
arrival_airport as arrival_airport_of_Ms_Ella_Dmitirieva
FROM Flights f, Ticket_flights tf, Tickets t
WHERE f.flight_id = tf.flight_id 
AND tf.ticket_no = t.ticket_no
AND passenger_name = 'ELLA DMITRIEVA'

7 rows affected.


departure_airport_of_ms_ella_dmitirieva,arrival_airport_of_ms_ella_dmitirieva
DME,HMA
HMA,DME
DME,UUS
SVO,SVX
SVX,SVO
SVX,MQF
MQF,SVX


In [5]:
%%sql $CONNECT_DATA
SELECT departure_airport as departure_airport_of_Ms_Ella_Dmitirieva, 
arrival_airport as arrival_airport_of_Ms_Ella_Dmitirieva
FROM Flights
JOIN Ticket_flights USING(flight_id)
JOIN Tickets USING(ticket_no)
WHERE passenger_name = 'ELLA DMITRIEVA'

7 rows affected.


departure_airport_of_ms_ella_dmitirieva,arrival_airport_of_ms_ella_dmitirieva
DME,UUS
SVO,SVX
SVX,SVO
SVX,MQF
MQF,SVX
DME,HMA
HMA,DME


In [ ]:
#WOW, she travelled to my hometown (Magnitogorsk, MQF) even!

In [ ]:
#Solution with JOINS

In [3]:
%%sql $CONNECT_DATA
SELECT 
passenger_name, 
ticket_no_of_ELLA, 
t.flight_id,
departure_airport as departure_airport_of_Ms_Ella_Dmitirieva, 
arrival_airport as arrival_airport_of_Ms_Ella_Dmitirieva
FROM Flights
JOIN ((SELECT passenger_name, a.ticket_no as ticket_no_of_ELLA, flight_id 
                    FROM Ticket_flights
                    JOIN (SELECT passenger_name, ticket_no 
                                        FROM Tickets
                                        WHERE passenger_name = 'ELLA DMITRIEVA') as a
                    ON Ticket_flights.ticket_no = a.ticket_no)) as t
ON Flights.flight_id = t.flight_id
                    

7 rows affected.


passenger_name,ticket_no_of_ella,flight_id,departure_airport_of_ms_ella_dmitirieva,arrival_airport_of_ms_ella_dmitirieva
ELLA DMITRIEVA,0005432284229,8202,DME,UUS
ELLA DMITRIEVA,0005434344563,34407,SVO,SVX
ELLA DMITRIEVA,0005433516547,11445,DME,HMA
ELLA DMITRIEVA,0005433516547,142605,HMA,DME
ELLA DMITRIEVA,0005434344563,70804,SVX,SVO
ELLA DMITRIEVA,0005434344563,74285,SVX,MQF
ELLA DMITRIEVA,0005434344563,193132,MQF,SVX
